ALL IN ONE IPYNB FILE

In [1]:
import gym
from gym import spaces

import pygame
import random
import numpy as np

import traceback

WIDTH, HEIGHT = 1005,  510
get_random_apple = lambda: [random.randrange(1,int(WIDTH/15))*15,random.randrange(1,int(HEIGHT/15))*15]

class Snake:

    def __init__(self):
        
        x = random.randint(30, WIDTH-50)
        x -= x % 15 or 0
        #x = (lambda num: num - num % 15 or 1)(random.randint(100, 1000))
        
        y = random.randint(100, HEIGHT-50)
        y -= y % 15 or 0
        
        self.head = [x,y]
        self.tail = [[x+15,y],[x+30,y],[x+45,y]]
        
        
class SnakeEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self):
        super(SnakeEnv, self).__init__()
        
        self.done = False #For episode 
        self.run = True # For game window
        
        self.action_space = spaces.Discrete(4) #For left right up and down
        self.observation_space = spaces.Box(low=0, high=1, shape=(5,), dtype=float) # 5 = Snake_X, Snake_Y, Apple_X, Apple_Y, Euclid_Distance_Apple_to_Snake
        self.info = {}
        
        self.Agent = Snake()
        self.Apple = get_random_apple()
        
        self.distance = np.linalg.norm(np.array([self.Agent.head[0], self.Agent.head[1]]) - np.array([self.Apple[0], self.Apple[1]]))
    
    
    def step(self, action):
    
        self.render()
        
        reward = 0
        
        if action == 0: #left
            self.Agent.head[0] -= 15
    
        if action == 1: #up
            self.Agent.head[1] -= 15
    
        if action == 2: #right
            self.Agent.head[0] += 15
    
        if action == 3: # down
            self.Agent.head[1] += 15
    
        if np.linalg.norm(np.array([self.Agent.head[0], self.Agent.head[1]]) - np.array([self.Apple[0], self.Apple[1]])) < self.distance:
            reward += 15
        
        else:
            reward -= 5
        
        self.distance = np.linalg.norm(np.array([self.Agent.head[0], self.Agent.head[1]]) - np.array([self.Apple[0], self.Apple[1]]))
        
        self.observation = np.array([self.Agent.head[0], self.Agent.head[1], self.Apple[0], self.Apple[1], self.distance])
        
        if self.Agent.head[1] < 0 or self.Agent.head[0] < 0 or self.Agent.head[1] > HEIGHT or self.Agent.head[0] > WIDTH:
            self.done = True
       
        if self.Apple == self.Agent.head:
            
            self.Agent.tail.insert(0,list(self.Agent.head))
            self.Apple = get_random_apple()
            self.distance = np.linalg.norm(np.array([self.Agent.head[0], self.Agent.head[1]]) - np.array([self.Apple[0], self.Apple[1]]))
                
        
        return self.observation, reward, self.done, self.info
        
    
    def reset(self):
    
        self.Agent = Snake()
        self.Apple = get_random_apple()
        
        self.distance = np.linalg.norm(np.array([self.Agent.head[0], self.Agent.head[1]]) - np.array([self.Apple[0], self.Apple[1]]))
    
        self.observation = np.array([self.Agent.head[0], self.Agent.head[1], self.Apple[0], self.Apple[1], self.distance])
        self.done = False
        
        return self.observation
    
    
    def render(self):
        
        screen.fill((0,0,0))
        pygame.draw.rect(screen, (255,0,0), [self.Apple[0],self.Apple[1], 15,15])

        for pos in self.Agent.tail:
            pygame.draw.rect(screen, (0,0,120), [pos[0], pos[1], 15,15])

        self.Agent.tail.insert(0,list(self.Agent.head))
        self.Agent.tail.pop()

        
        blockSize = 15 
        for x in range(0, 1200, blockSize):
            for y in range(75, 800, blockSize):
                rect = pygame.Rect(x, y, blockSize, blockSize)
                pygame.draw.rect(screen, (25,25,25), rect, 1)
        
        pygame.display.update()
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                self.run = False
                self.close()

In [2]:
env = SnakeEnv()
screen = pygame.display.set_mode((WIDTH, HEIGHT))

episode = 10

while episode > 0 and env.run:
    state = env.reset()
    env.done = False
    score = 0
    
    while not env.done and env.run:
     
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
        
    print('Episode: {} Score:{}'.format(11 - episode , score))

pygame.display.quit()
pygame.quit()

Episode: 1 Score:2790


In [ ]:
from stable_baselines3 import PPO, A2C
import traceback
import sys


WIDTH, HEIGHT = 1005,  510
screen = pygame.display.set_mode((WIDTH,HEIGHT))


env = SnakeEnv()

try:
    model = A2C('MlpPolicy',env, learning_rate=0.0004, gamma=0.99, gae_lambda=0.95, max_grad_norm=0.5, verbose=0)
    
    #policy, env, learning_rate=0.0003, n_steps=2048, batch_size=64, n_epochs=10, gamma=0.99, gae_lambda=0.95, clip_range=0.2, 
    #clip_range_vf=None, normalize_advantage=True, ent_coef=0.0, vf_coef=0.5, max_grad_norm=0.5, use_sde=False, sde_sample_freq=-1, 
    #target_kl=None, stats_window_size=100, tensorboard_log=None, policy_kwargs=None, verbose=0, seed=None, device='auto', _init_setup_model=True
    
    model.learn(total_timesteps=500000, callback=None, log_interval=1, tb_log_name='PPO', reset_num_timesteps=True, progress_bar=False)
    
    """
    gamma = A higher gamma value (e.g., 0.99) means that the agent considers future rewards more heavily, which can lead to a longer-term focus in its learning. Conversely, a lower gamma value (e.g., 0.9) makes the agent prioritize immediate rewards.
    gae_lambda = is the factor used for trade-off between bias and variance when estimating advantages with the Generalized Advantage Estimator (GAE). GAE is used to estimate how much better or worse taking a specific action is compared to following the current policy. A higher gae_lambda value (e.g., 0.95) reduces bias but may increase variance, while a lower value reduces variance but may introduce bias into the advantage estimates.
    max_grad_norm = is the maximum allowed value for gradient clipping during the training process. Gradient clipping is a technique used to prevent exploding gradients, which can lead to unstable training. If the magnitude of the gradients exceeds this value, they are scaled down to ensure they do not become too large.
    """
    
except Exception as e:
    exc_type, exc_obj, tb = sys.exc_info()
    line_number = tb.tb_lineno
    print("!Error at", line_number,"th Line")
    traceback.print_exc()
    pygame.quit()
    

C:\Users\gokss\anaconda3\envs\mech\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
